In [1]:
import os
import ray
import supersuit as ss 
from ray import tune    # experiment runner
from pettingzoo.butterfly import cooperative_pong_v5
from ray.tune.registry import register_env
from ray.rllib.env import ParallelPettingZooEnv
from ray.rllib.algorithms.ppo import PPOConfig

ray.shutdown()   # restart ray
ray.init(num_cpus=6)

2024-10-31 15:35:54,973	INFO worker.py:1816 -- Started a local Ray instance.


Python version:,3.11.6
Ray version:,2.38.0


# Ray, RLLib and Pettingzoo
To perform RL experiments with Ray, four things are needed: 
1. RL Environment 
2. RL Algorithm
3. Configuration of the environment, algorithm and the experiment
4. Experiment Runner


The multi-agent environments from PettingZoo are not directly compatible with Ray, and need to be wrapped with either the ```PettingZooEnv``` or the ```PettingZooParallelEnv``` wrappers, depending on whether an ```AEC``` or a ```ParallelEnv``` environment is being used. In the following the ```ParallelEnv``` option will be shown using the ```cooperative_pong_v5```.

In [2]:
env_name = 'cooperative_pong'

def env_creator(env_config):
    env = cooperative_pong_v5.parallel_env(render_mode=env_config.get("render_mode", "human"))
    env = ss.color_reduction_v0(env, mode='B')
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, 4)
    env = ss.dtype_v0(env, 'float32')
    return env

register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

In the ```env_creator``` function we initialise the environment and using ```Supersuit``` we wrap the environment to be compatible with the ```rllib``` algorithms. The single functions are described [here](https://pypi.org/project/SuperSuit/3.3.1/). 

By registering the environment under the ```env_name``` we can access it from the ```rllib``` API. When the env is called, it invokes the ```env_creator``` function defined above and passes the arguments in ```config```. The ```.get()``` function retrieves the values for the key, allowing the specification of a default value if that key doesn't exist in the dictionary.

## Algorithm and AlgorithmConfig

We pass all important configuration setting to the ```PPOConfig``` class and run an experiment using a ``tune.Tuner`` object. 

, two variants of configuring and running an algorithm. One method is to set all configuration settings in the ```config``` object directly when accessing ```PPOConfig``` and run the configuration using ```tune.run()```.

In [6]:
config = (
    PPOConfig()
    .environment(env=env_name)
    .framework("torch")
    .training(
            train_batch_size=512,
            lr=2e-5,
            gamma=0.99,
            lambda_=0.9,
            use_gae=True,
            clip_param=0.4,
            grad_clip=None,
            entropy_coeff=0.1,
            vf_loss_coeff=0.25,
            num_sgd_iter=10,
    )
)



2024-10-31 16:46:10,677	WARNING deprecation.py:50 -- DeprecationWarning: `config.training(num_sgd_iter=..)` has been deprecated. Use `config.training(num_epochs=..)` instead. This will raise an error in the future!


## Ray Train vs. Tune
When a use-defined training function is desired, using the ``Trainer`` from the ``ray.train`` module can be useful, as it allows for custom training logic, model and dataset loading, training loop, checkpoint saving and logging metrics. In our case, no custom training function is necessary, so we can allow ``ray.tune.run()`` to handle the training logic for us. 

We take the previously defined configuration, passing the name of our experiment and set stop criterion based on the average episode reward and overall steps. 

In [10]:
results = tune.run(
               'PPO',
               config=config.to_dict(),
               name='ppo_cooperative_pong',
               stop={'agent_timesteps_total': 1e4, 'env_runners/episode_reward_mean': 10},
)

2024-10-31 16:52:54,332	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,agent_timesteps_total,counters,custom_metrics,episode_media,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_cooperative_pong_30efb_00000,13312,"{'num_env_steps_sampled': 6656, 'num_env_steps_trained': 6656, 'num_agent_steps_sampled': 13312, 'num_agent_steps_trained': 13312}",{},{},"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(44.679874), 'cur_kl_coeff': np.float64(5.125781250000001), 'cur_lr': np.float64(2e-05), 'total_loss': np.float64(0.19177034539170562), 'policy_loss': np.float64(-0.07176895532757044), 'vf_loss': np.float64(1.2496330261230468), 'vf_explained_var': np.float64(0.7856563657522202), 'kl': np.float64(0.006372157076655965), 'entropy': np.float64(0.8153123579919338), 'entropy_coeff': np.float64(0.09999999999999999)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(128.0), 'num_grad_updates_lifetime': np.float64(1000.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(39.5)}}, 'num_env_steps_sampled': 6656, 'num_env_steps_trained': 6656, 'num_agent_steps_sampled': 13312, 'num_agent_steps_trained': 13312}",13312,13312,13312,6656,6656,512,9.64331,6656,512,9.64331,2,0,0,512,"{'cpu_util_percent': np.float64(5.902666666666666), 'ram_util_percent': np.float64(62.65866666666666)}","{'training_iteration_time_ms': 49458.651, 'restore_workers_time_ms': 0.0, 'training_step_time_ms': 49458.651, 'sample_time_ms': 34466.41, 'load_time_ms': 3.835, 'load_throughput': 133515.105, 'learn_time_ms': 14979.38, 'learn_throughput': 34.18, 'synch_weights_time_ms': 8.725}"


2024-10-31 17:04:06,724	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-10-31 17:04:06,742	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/ushe/ray_results/ppo_cooperative_pong' in 0.0171s.
2024-10-31 17:04:16,859	INFO tune.py:1041 -- Total run time: 682.53 seconds (672.38 seconds for the tuning loop).
2024-10-31 17:04:16,860	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
